## В этом домашнем задании вы сделаете первые шаги в мире линейной бинарной классификации!

In [425]:
import pandas as pd
import numpy as np
import warnings

#### Задание 1

Мы будем работать с данными **Microsoft Malware Detection**

Таргетом будет последний столбец `HasDetection`, который принимает значения $\{0,\, 1\}$ в случае отсутствия или наличия вируса на компьютере соответственно. Признаками будут выступать всевозможные характеристики компьютера.

In [426]:
data = pd.read_csv('train.csv')

C:\Users\jackt\AppData\Local\Temp\ipykernel_14568\2815211952.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('train.csv')


In [427]:
data.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


Удалите константные признаки и признаки `ProductName` `MachineIdentifier`

In [428]:
data = data.drop(['ProductName', 'MachineIdentifier'], axis = 1)
categorical_columns = data.loc[:, data.dtypes == object].columns

In [429]:
for col in data.columns:
    if (data[col]==data[col][0]).all():
        print(col)
        data.drop(columns = [col], inplace = True)

IsBeta


Посмотрите на соотношение классов в таргете. Все ли хорошо?

In [430]:
print(sum(data['HasDetections'] == 1), '- positive class,')
print(sum(data['HasDetections'] == 0), '- negative class')

100060 - positive class,
99940 - negative class


Ответьте на вопрос: почему с вашей точки зрения важно иметь представление о балансе классов в ваших данных?

Избавьтесь от пропусков в данных! 

Новый для нас прием: признаки с более чем половиной пропусков следует удалить.

Согласитесь, если в вашей колонке среди 100 объектов всего лишь у 2 есть какое-то непропущенное значение, странно все остальные заполнять средним от этих двух чисел. Такие "редкие" признаки лучше вообще опустить!


В категориальных колонках заменим отсутствующую категорию просто некоторой новой и назовем ее `NaN`.

А в числовых, ради разнообразия, заполним пропуски медианным значением.

In [431]:
for col in data.columns:
    if data[col].isna().sum()>data.shape[0]/2:
        data = data.drop(col, axis = 1)
        print(col)

DefaultBrowsersIdentifier
PuaMode
Census_ProcessorClass
Census_InternalBatteryType
Census_IsFlightingInternal
Census_ThresholdOptIn
Census_IsWIMBootEnabled


In [432]:
### В категориальных колонках с отсутствующими значениями
### произведем fillna, создав по новой категории для каждой
### такой фичи

cat_cols = data.select_dtypes(exclude=['float64', 'int64']).isnull().any()
for null_col in cat_cols[cat_cols].index:
    data[null_col] = data[null_col].fillna('NaN')


num_cols = data.select_dtypes(include=['float64', 'int64']).isnull().any()
for null_col in num_cols[num_cols].index:
    mean_value = data[null_col].median()
    data[null_col] = data[null_col].fillna(mean_value)

Создайте копию полученного датафрейма и положите ее в переменную data_2. Понадобится в следующих заданиях.

In [433]:
data_2 = data.copy()

In [434]:
data_2.isna().sum()

EngineVersion                              0
AppVersion                                 0
AvSigVersion                               0
RtpStateBitfield                           0
IsSxsPassiveMode                           0
                                          ..
Census_IsPenCapable                        0
Census_IsAlwaysOnAlwaysConnectedCapable    0
Wdft_IsGamer                               0
Wdft_RegionIdentifier                      0
HasDetections                              0
Length: 73, dtype: int64

Так же поработаем над всеми категориальными колонкам перед запуском непосредственно моделей.

Провернем самый базовый и наглый метод - несмотря на количество уникальных значений в каждой категории, просто применим ко всей категориальной части датасета `OneHotEncoding`

In [435]:
data.shape

(200000, 73)

In [436]:
### Для OneHotEncoding'а воспользуемся пандасовским get_dummies

cat_cols = data.select_dtypes(exclude=['float64', 'int64']).columns

for col in cat_cols:
    one_hot = pd.get_dummies(data[col], prefix = col, drop_first = True)
    data = pd.concat((data.drop(col, axis = 1), one_hot),axis = 1)

In [437]:
data.shape[1]

6114

###  Разделим выборку на тренировочную и тестовую

P.S. в задачах классификации, как и в задаче регрессии, можно использовать технологию Кросс-Валидации. Например, по одному из двух следующих сценариев:

1) Отделить валидацию и тест, произвести подбор лучшей модели с помощью `K-Fold` на валидации, финально обучить выбранную модель на всей валидации и замерить качество на заранее отложенном финальном тесте!

2) Всю выборку назвать валидационной и на ней применить `K-Fold` без финального замера.

В этой домашней работе попросим Вас быть еще проще! :)
Реализуем просто технологию отложенной выборки в пропорции 3:1

In [438]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['HasDetections'])
y = data['HasDetections']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.25,
                                                   shuffle = True, 
                                                   random_state = 1)

Соберите `Pipeline`, реализовав в нем 2 шага: стандартизация данных через `StandardScaler` и обучение логистической регрессии с помощью `LogisticRegression`, положите результаты в переменную `pipe`, а в классе модели `LogisticRegression` укажите параметр `penalty='none'`

In [439]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([('scaler', StandardScaler()),
                ('LR', LogisticRegression(penalty = None))])

In [440]:
from sklearn.model_selection import cross_validate
import datetime

### Создадим такой список из одного единственного кортежа
### Первый элемент - индексы обучающей выборки
### Второй элемент - индексы тестовой выборки

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

Чтобы замерить качество работы такой модели на трейне и на тесте воспользуемся функцией `cross_validate`

Вопрос: что передавать ей в параметр cv? Ведь мы уже разделили нашу выборку на трейн и тест (имеем всего 1 fold). Для этого можно просто передать список от кортежа, содержащего индексы тренировочных и тестовых объектов.

Что можете сказать про время работы алгоритма?

Очевидно, оно достаточно большие. Уж тем более для линейных моделей.

Такое как раз-таки происходит из-за того, что количество фичей, который мы передали нашей модели - гигантское! Классу требуется много времени и памяти, чтобы обработать датасет.

Поэтому те колонки, в которых количество уникальных категорий превышает какое-то адекватное число, следует кодировать иначе, нежели с помощью технологии `One-Hot-Encoding`.

Теперь вы верите, что более умные кодировки зачастую прям необходимы! Раньше мы этот факт не демонстрировали!

Дело еще вот в чем: в классе `LogisticRegression`, как и, например, `Lasso`, есть параметр, ограничивающий максимальное количество итераций во время обучения модели. Так, если данных много и итераций тоже ожидается большое число, найденная разделяющая гиперплоскость может оказаться не самой лучшей, так как наш алгоритм (будь то градиентный спуск или любой иной) просто 'не доползет'. 

#### Задание 2

Теперь попробуем другой метод кодирования категориальных колонок, а именно счётчики.
Построем ту же модель и на том же разделении, просто заново иначе переобработаем датасет. 

Для тех категориальных признаков, у которых количество уникальных значений в колоночках больше 5, применим `MeanTargetEncoding`.

Для остальных оставим любимый `OneHotEncoding` (как делали на практике и в предыдущем уроке).

In [441]:
### Your code is here
categorical_columns = data_2.select_dtypes(exclude = ['float64', 'int64']).columns
for col in categorical_columns:
    if data_2[col].nunique()<=5:
        one_hot = pd.get_dummies(data_2[col], prefix = col, drop_first = True)
        data_2 = pd.concat((data_2.drop(col, axis = 1), one_hot), axis = 1)
    else:
        mean_target = data_2.groupby(col)['HasDetections'].mean()
        data_2[col] = data_2[col].map(mean_target)

In [442]:
data_2 = data_2*1

In [443]:
data_2.isna().sum()

EngineVersion                         0
AppVersion                            0
AvSigVersion                          0
RtpStateBitfield                      0
IsSxsPassiveMode                      0
                                     ..
Census_OSArchitecture_arm64           0
Census_OSArchitecture_x86             0
Census_GenuineStateName_IS_GENUINE    0
Census_GenuineStateName_OFFLINE       0
Census_GenuineStateName_UNKNOWN       0
Length: 82, dtype: int64

In [444]:
X_2 = data_2.drop(columns=['HasDetections'])
y_2 = data_2['HasDetections']

Опять обучим модель, пока что без изменений! Скажите, стало ли быстрее? А что с качеством?

In [445]:
### Your code is here
from sklearn.model_selection import KFold

pipe = Pipeline([('scaler', StandardScaler()),
                ('LR', LogisticRegression(penalty = None))])

cv_result = cross_validate(pipe, X_2, y_2, scoring = 'accuracy',
                          cv = custom_cv, return_train_score = True)

print(cv_result)



{'fit_time': array([1.83644223]), 'score_time': array([0.05295372]), 'test_score': array([0.63768]), 'train_score': array([0.63796])}


In [446]:
print(f'test score is {round(np.mean(cv_result['test_score']),3)}')
print(f'train score is {round(np.mean(cv_result['train_score']),3)}')

test score is 0.638
train score is 0.638


#### Задание 3: Регуляризация

Как и в моделях регрессии, решая задачу классификации, можем штрафовать минимизируемый функционал за большие веса, добавив к нему L1 или L2 норму весов (все как раньше!).

Для этого в изначальном классе `LogisticRegression` изменить параметр `penalty` на l1 или l2 соответственно. Выберите второй вариант! Можно воспользоваться методом `set_params` и применить его к `pipe`.

In [450]:
### Your code is here
pipe.set_params(LR__penalty = 'l2')

Pipeline(steps=[('scaler', StandardScaler()), ('LR', LogisticRegression())])

Теперь наша модель будет строить логистическую регрессию с L2 регуляризатором! Помним, что у регуляризируемых моделей есть гиперпараметр, контролирующий силу регуляризации, который выбирается ДО запуска метода fit, то есть заранее, когда модель еще не обучена. 

Конечно же, выбор этого параметра влияет итоговые результаты. Хочется поперебирать различные параметры регуляризации и найти такой, при котором качество на тесте окажется лучшим! 

Сгенерируем массив гиперпараметра, которые планируем перебрать:

In [451]:
alphas = np.linspace(0.01, 100, 100)
alphas

array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+01, 2.223e+01, 2.324e+01,
       2.425e+01, 2.526e+01, 2.627e+01, 2.728e+01, 2.829e+01, 2.930e+01,
       3.031e+01, 3.132e+01, 3.233e+01, 3.334e+01, 3.435e+01, 3.536e+01,
       3.637e+01, 3.738e+01, 3.839e+01, 3.940e+01, 4.041e+01, 4.142e+01,
       4.243e+01, 4.344e+01, 4.445e+01, 4.546e+01, 4.647e+01, 4.748e+01,
       4.849e+01, 4.950e+01, 5.051e+01, 5.152e+01, 5.253e+01, 5.354e+01,
       5.455e+01, 5.556e+01, 5.657e+01, 5.758e+01, 5.859e+01, 5.960e+01,
       6.061e+01, 6.162e+01, 6.263e+01, 6.364e+01, 6.465e+01, 6.566e+01,
       6.667e+01, 6.768e+01, 6.869e+01, 6.970e+01, 7.071e+01, 7.172e+01,
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 

Чтобы отобрать среди данного массива гиперпараметров лучший, воспользуйтесь конструкцией `GridSearchCV` из `sklearn`

In [453]:
from sklearn.model_selection import GridSearchCV

param_grid ={
    'LR__C': alphas
}

search = GridSearchCV(pipe, param_grid,
                     cv = custom_cv, scoring = 'accuracy')

search.fit(X_2, y_2)

print(search.best_params_)
### Your code is here


{'LR__C': np.float64(0.01)}


In [455]:
pipe.set_params(LR__C = 0.01)

cv_result_new = cross_validate(pipe, X_2, y_2, scoring = 'accuracy',
                              cv = custom_cv, return_train_score = True)
print(f'test score is {round(np.mean(cv_result_new['test_score']),3)}')
print(f'train score is {round(np.mean(cv_result_new['train_score']),3)}')

test score is 0.638
train score is 0.638


#### Задание 4: Бонус

Как вы знаете, подбор признаков является одной из самых важных частей машинного обучения. Сейчас вы попробуете на основе имеющихся признаков сгенерировать новые. В качестве новых признаков будем использовать мономы 2 степени. Можно использовать регуляризацию различного рода, выбор энкодера тоже за вами. Ваша задача - добиться качества `0.65` на тестовой выборке

In [19]:
### Your code is here

